In [ ]:
import pandas as pd
from openpyxl import load_workbook
import glob
from pathlib import Path
pd.set_option('display.expand_frame_repr', False)  # Evita que las filas se dividan
pd.set_option('display.max_columns', None)  # Muestra todas las columnas


In [ ]:
wb = load_workbook('config.xlsx')
ws = wb.active
valor = ws.cell(row=3, column=2).value

carpeta = Path(valor) 
archivo = list(carpeta.rglob("CRM*.xlsx")) + list(carpeta.glob("CRM*.xls")) # se filtra por CRM xlsx u xls
archivos_str = [str(a) for a in archivo] # retorna un array ['xxx','xxxx',...]de las ruta de los archivos

In [ ]:
def leer_excel_smart(path, hoja_principal="Base CRM", hoja_backup="Hoja1"):
    """
    Lee un Excel detectando:
    - qué hoja usar (principal o backup)
    - primera fila válida como encabezado
    - limpia filas vacías
    - estandariza nombres de columnas
    """

    column_map = {
        "No.": "numero",
        "No": "numero",
        "N°": "numero",
        "nro": "numero",

        "Fecha": "fecha",
        "Asesor": "asesor",
        "Contacto": "contacto",
        "Cargo": "cargo",

        "teléfono": "telefono",
        "telefono": "telefono",
        "Teléfono": "telefono",

        "Mail": "mail",
        "Correo": "mail",

        "Razón Social": "razon_social",
        "Razon Social": "razon_social",

        "RUC": "ruc",
        "Web": "web",
        "Departamento": "departamento",
        "Distrito": "distrito",
        "Rubro": "rubro",
        "Tipo de producto": "tipo_producto",
        "Tipo de Producto": "tipo_producto",

        "Origen del lead": "origen_lead",
        "Descripción": "descripcion",
        "Estado": "estado",
        "Monto de la oportunidad": "monto_oportunidad",
        "No. Cotización": "numero_cotizacion",

        "Valor Cotización": "valor_cotizacion",
        "Valor Cotización + IGV": "valor_cotizacion",

        "Fecha seguimiento": "fecha_seguimiento",
        "Semana": "semana",
    }

    try:
        xls = pd.ExcelFile(path)
    except Exception as e:
        print(f"❌ Error al abrir archivo {path}: {e}")
        return None

    if hoja_principal in xls.sheet_names:
        hoja = hoja_principal
    elif hoja_backup in xls.sheet_names:
        hoja = hoja_backup
    else:
        print(f"❌ Ninguna hoja válida encontrada en: {path}")
        return None

    df_raw = pd.read_excel(path, sheet_name=hoja, header=None)

    header_row = next((i for i in range(len(df_raw)) if df_raw.iloc[i].notna().any()), None) # busca la primera fila que no esta vacia

    if header_row is None:
        print(f"❌ No se encontró encabezado en: {path}")
        return None

    df = pd.read_excel(path, sheet_name=hoja, header=header_row)

    df = df.dropna(how='all').reset_index(drop=True) # quita las fila completamente vacias

    # Estandarizar columnas
    df.columns = df.columns.astype(str).str.strip()  # quita espacios
    
    df = df.rename(columns=lambda c: column_map.get(c, c.lower().replace(" ", "_")))
    
    return df



In [ ]:
archivo_lectura = []
archivo_cierre = []
for archivo  in archivos_str:
    df_1 = leer_excel_smart(archivo, hoja_principal="Base CRM", hoja_backup="CRM")
    if df_1 is None:
        print(f"Saltando archivo no válido: {archivo}")
        continue
    archivo_lectura.append(df_1)
    
    

In [ ]:
df_fina = pd.concat(archivo_lectura, ignore_index=True) # creamos un dataframe 

In [ ]:
df_fina.head(10)

In [ ]:
df_fina.info()

In [ ]:
with pd.ExcelWriter("salida.xlsx", engine="openpyxl") as writer:
    df_fina.to_excel(writer, index=False)
    wb = writer.book
    ws = writer.sheets['Sheet1']
    
    for col in ['B','U']:
        for cell in ws[col]:
            if cell.value is not None:
                cell.number_format = "DD/MM/YYYY"
            
    for col in ['D','E','F']:
        for cell in ws[col]:
            if cell.value is not None:
                cell.number_format = "@"
              
    

In [ ]:
# desarrollo de cierre
datosCierre = []
for archivo in archivos_str:
    # desarrollo de cierre
    claves = ['No.','Fecha','Asesor','Razon Social','RUC']
    pattern = "|".join(claves)
    dfCierre = pd.read_excel(archivo, sheet_name='CIERRE', header=None)

    # Buscar fila donde están los encabezados
    filaCierre = dfCierre.index[
        dfCierre.apply(lambda row: row.astype(str).str.contains(pattern, case=False).any(), axis=1)
    ][0]

    print("Fila encontrada:", filaCierre)

    # Cargar excel usando esa fila como encabezado
    df_final = pd.read_excel(
        archivo,
        sheet_name='CIERRE',
        header=filaCierre
    )

    datosCierre.append(df_final)
    
dfCierres = pd.concat(datosCierre, ignore_index=True)
print(dfCierres)

In [ ]:
import pandas as pd
import glob
import os

ruta = "Codigos Costos"
codigo_buscar = "I22900"

archivos = glob.glob(os.path.join(ruta, "*.xlsx"))

# Lista donde guardaremos cada fila encontrada
filas_encontradas = []

for archivo in archivos:
    try:
        xls = pd.ExcelFile(archivo)

        for hoja in xls.sheet_names:
            df = pd.read_excel(archivo, sheet_name=hoja)

            # Buscar filas donde exista el valor en cualquier columna
            mask = df.isin([codigo_buscar]).any(axis=1)
            filas = df[mask]

            if not filas.empty:
                # Agregar metadatos útiles
                filas["archivo"] = os.path.basename(archivo)
                filas["hoja"] = hoja

                filas_encontradas.append(filas)

    except Exception as e:
        print(f"Error leyendo {archivo}: {e}")

# Unir todas las coincidencias en un solo DataFrame
if filas_encontradas:
    resultado = pd.concat(filas_encontradas, ignore_index=True)
    print("✔ FILAS ENCONTRADAS:")
    print(resultado)
else:
    print("❌ No se encontró el código en ningún archivo.")